In [1]:
from datetime import datetime, date
import requests
import pandas as pd

In [ ]:
def fetch_spot_data(start: date, end: date) -> pd.DataFrame:
    
    """Fetches spot market price data from the aWATTar API for a given date range."""
    base_url = "https://api.awattar.at/v1/marketdata"
    start_dt, end_dt = datetime.combine(start, datetime.min.time()), datetime.combine(end + pd.Timedelta(days=1), datetime.min.time())
    params = {"start": int(start_dt.timestamp() * 1000),
              "end": int(end_dt.timestamp() * 1000)}
    
    print(f"{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}: Will fetch data from Awattar.")

    response = requests.get(base_url, params=params)
    response.raise_for_status()

    df = pd.DataFrame(response.json()["data"])
    df["timestamp"] = pd.to_datetime(df["start_timestamp"], unit="ms", utc=True)
    df["spot_price_eur_kwh"] = df["marketprice"] / 1000 * 1.2 # Convert from Eur/MWh to c/kWh and add VAT
    return df[["timestamp", "spot_price_eur_kwh"]]

In [ ]:
# Gleiche Preise wie hier: https://www.energymarket.solutions/day-ahead-borsenpreise/ -> Deutschland?
df = pd.DataFrame(requests.get("https://api.awattar.at/v1/marketdata").json()["data"])

In [13]:
df

,start_timestamp,end_timestamp,marketprice,unit
0,1753783200000,1753786800000,25.60,Eur/MWh
1,1753786800000,1753790400000,4.34,Eur/MWh
2,1753790400000,1753794000000,1.07,Eur/MWh
3,1753794000000,1753797600000,6.20,Eur/MWh
4,1753797600000,1753801200000,44.41,Eur/MWh
5,1753801200000,1753804800000,78.24,Eur/MWh
6,1753804800000,1753808400000,94.81,Eur/MWh
7,1753808400000,1753812000000,105.32,Eur/MWh
8,1753812000000,1753815600000,123.56,Eur/MWh
9,1753815600000,1753819200000,126.01,Eur/MWh


In [15]:
df["timestamp"] = pd.to_datetime(df["start_timestamp"], unit="ms", utc=True)

In [19]:
df["marketprice"]

0      25.60
1       4.34
2       1.07
3       6.20
4      44.41
5      78.24
6      94.81
7     105.32
8     123.56
9     126.01
10    111.94
11    102.07
Name: marketprice, dtype: float64